# Training notebook

This file is part of a decoder for small stabilizer codes, in particular color and surface codes, based on a combination of recurrent and feedforward neural networks.

Copyright 2018 Paul Baireuther. All Rights Reserved.

# Parameters to be set by user

In [ ]:
# # # FILES AND DIRECTORIES # # #

# Set path to source files of this project
src_path =  # "../src/"

# Set path where checkpoints, models, and feedback will be stored
# WARNING: Existing files in this directory may be overwritten
checkpoint_path =  # "../checkpoints/dist3/"

# Set path to the databases that contain the datasets
data_path =  # "../data/color_666_dist_3/"

# Set path and filenames of databases for training and validation
training_db_fname =  # data_path + "colorcode_distance_3_train_p_0.100__canonical.db"
validation_db_fname =  # data_path + "colorcode_distance_3_validation_p_0.010__canonical.db"

In [ ]:
# # # PARAMETERS # # #

# Network (parameters for 6-6-6 color code with distance 3)
code_distance = 3  # distance of the QEC code
dim_syndr = 12  # dimension of the syndrome increment vectors
dim_fsyndr = 3  # dimension of the final syndrome increment vectors
network_size = 32  # size of the LSTM's internal states and number of the the FF layers' neurons

# Network (parameters for 6-6-6 color code with distance 5)
# code_distance = 5  # distance of the QEC code
# dim_syndr = 36  # dimension of the syndrome increment vectors
# dim_fsyndr = 9  # dimension of the final syndrome increment vectors
# network_size = 64  # size of the LSTM's internal states and number of the the FF layers' neurons

# Network (parameters for 6-6-6 color code with distance 7)
# code_distance = 7  # distance of the QEC code
# dim_syndr = 72  # dimension of the syndrome increment vectors
# dim_fsyndr = 18  # dimension of the final syndrome increment vectors
# network_size = 128  # size of the LSTM's internal states and number of the the FF layers' neurons


# Training (example parameters)
lr = 0.001  # learning rate
keep_prob = 0.8  # keep probability during dropout
aux_loss_factor = 0.5  # weight of auxiliary loss term
l2_prefactor = 10**(-5)  # prefactor for L2 regularization of weights

# Specify the maximum number of stabilizer measurement cycles in training and validation datasets
max_len_train_sequences = 40  # the maximum number of cycles in the training dataset
max_len_validation_sequences = 10000  # the maximum number cycles in the validation dataset

# Other hyperparameters
batch_size_training = 64  # batch-size for training
no_batches_feedback = 10  # number of batches for feedback
batch_size_feedback = 100  # batch-size for feedback

In [ ]:
# # # TRAINING PIPELINE # # #

# Parameters are as follows:
# 1. minimum number of stabilizer measurement cycles of training sequences (None means all sequences in dataset can be used)
# 2. maximum number of stabilizer measurement cycles of training sequences (None means all sequences in dataset can be used)
# 3. number of epochs
# 4. batches per epoch
train_pipeline = []
train_pipeline.append((1,  5, 10, 3000))
train_pipeline.append((1, 10, 20, 5000))
train_pipeline.append((1, 20, 30, 5000))
train_pipeline.append((1, 30, 40, 5000))
train_pipeline.append((None, None, 900, 5000))

# Main part of the notebook

In [ ]:
# # # LIBRARIES # # #

# Third party libraries
import os
import sys
import numpy as np

# This project
sys.path.insert(0, src_path);
import decoder as dec
import database_io as qec_db
import qec_functions as fcts

In [ ]:
# # # CHECKPOINTS # # #

# The training algorithm stores intermediate and final checkpoints 
# of the decoder during training, as well as feedback                               
print("cp path is", checkpoint_path)

# Check if the checkpoint_path exists
assert(os.stat(checkpoint_path))

# Check if subfolder "model" already exists and create it if it does not exist
try:
    os.stat(checkpoint_path + "model")
except:
    os.mkdir(checkpoint_path + "model")

In [ ]:
# # # DATABASES  # # #

# Check if data_path exists
assert(os.stat(data_path))

# Load databases
db = qec_db.Data(training_fname=training_db_fname,
                 validation_fname=validation_db_fname,
                 test_fname=None,
                 verbose=True,
                 store_in_memory=False)

In [ ]:
# Initialize decoder
net = dec.Decoder(code_distance=code_distance,
                  dim_syndr=dim_syndr,
                  dim_fsyndr=dim_fsyndr,
                  lstm_iss=[network_size, network_size],
                  ff_layer_sizes=[network_size],
                  checkpoint_path=checkpoint_path,
                  keep_prob=keep_prob,
                  aux_loss_factor=aux_loss_factor,
                  l2_prefactor=l2_prefactor)

# Start TensorFlow session
net.start_session()

In [ ]:
# # # Execute training pipeline # # #

# In case the training was only stopped, keep using the current best logical error rate plog_best
try:
    print("currently the best logical error rate was", plog_best)
except:
    plog_best = 5.0

for episode in train_pipeline:
    lmin_train, lmax_train, n_epochs, training_batches = episode
    
    for _ in range(n_epochs):
        epoch_idx = net.total_trained_epochs + 1
        print("epoch #" + str(epoch_idx))
        
        # Train for one epoch
        train_batches = db.gen_batches(n_batches=training_batches,
                                       batch_size=batch_size_training,
                                       db_type="training",
                                       len_buffered=max_len_train_sequences,
                                       len_min=lmin_train,
                                       len_max=lmax_train,
                                       select_random=True)
        net.train_one_epoch(train_batches=train_batches,
                            learning_rate=lr)
        
        # Generate feedback using training dataset
        train_batches = db.gen_batches(n_batches=no_batches_feedback,
                                       batch_size=batch_size_feedback,
                                       db_type="training",
                                       len_buffered=max_len_train_sequences,
                                       len_min=lmin_train,
                                       len_max=lmax_train,
                                       select_random=False)
        plog_train = net.calc_feedback(batches=train_batches,
                                       validation=False)
        
        # Generate feedback using validation dataset
        validation_batches = db.gen_batches(n_batches=no_batches_feedback, 
                                            batch_size=batch_size_feedback,
                                            db_type="validation",
                                            len_buffered=max_len_validation_sequences,
                                            len_min=None,
                                            len_max=None, 
                                            select_random=False)
        plog_val = net.calc_feedback(batches=validation_batches,
                                     validation=True)
        
        # Check if there is a new record logical error rate on the validation dataset
        if plog_val < plog_best:
            plog_best = plog_val
            print("--> new record logical error rate is", round(plog_best, 4))
            net.save_network("best")
        
        # Save history of logical error rates on validation dataset
        try:
            plog_history = np.loadtxt(net.cp_path + "plog_history.dat").reshape([-1, 2])
            plog_history = np.concatenate((plog_history, np.array([[epoch_idx, plog_val]])), axis=0)
        except:
            plog_history = np.array([[epoch_idx, plog_val]])
        np.savetxt(net.cp_path + "plog_history.dat", plog_history)